In [ ]:
import os
import sys
from multiprocessing.pool import ThreadPool

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

from av_data_process.YT_video_downloader2 import download, VidInfo

In [ ]:
def time_to_seconds(time: str):
    minutes, seconds = time.split(':') if ':' in time else (time, 0)
    # print(minutes, seconds)
    return int(minutes) * 60 + int(seconds)
    

In [ ]:
split = 'data/downloaded' # name of the destination folder
csv_file = '/Data/dec/DOLOS/dolos_timestamps.csv'
out_dir = split

os.makedirs(out_dir, exist_ok=True)

with open(csv_file, 'r') as f:
    lines = f.readlines()
    lines = [x.split(',') for x in lines]
    lines = lines[1:]

    # yt_id, file_name, start_time, end_time
    vidinfos = [VidInfo(x[0], x[1], time_to_seconds(x[2]), time_to_seconds(x[3]), out_dir) for x in lines]

bad_files = open('bad_files_{}.txt'.format(split), 'w')
results = ThreadPool(5).imap_unordered(download, vidinfos)
cnt = 0
for r in results:
    cnt += 1
    print(cnt, '/', len(vidinfos), r)
    if 'ERROR' in r:
        bad_files.write(r + '\n')
bad_files.close()

In [ ]:
import os
import sys
from multiprocessing.pool import ThreadPool

# Make sure your fixed YT_video_downloader2.py is in this path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

# Import the fixed version
from av_data_process.YT_video_downloader2 import download, VidInfo

def time_to_seconds(time_str):
    """Convert time string to seconds"""
    if not time_str or time_str.strip() == '':
        return 0
        
    # Handle simple seconds format
    if time_str.strip().replace('.', '').isdigit():
        return float(time_str)
    
    # Handle MM:SS format
    parts = time_str.strip().split(':')
    if len(parts) == 2:
        minutes, seconds = parts
        return int(minutes) * 60 + float(seconds)
    elif len(parts) == 3:
        hours, minutes, seconds = parts
        return int(hours) * 3600 + int(minutes) * 60 + float(seconds)
    else:
        # Try to convert directly
        try:
            return float(time_str)
        except ValueError:
            print(f"Warning: Could not parse time '{time_str}', using 0")
            return 0


In [ ]:

# Configure your paths
split = 'data/downloaded' # name of the destination folder
csv_file = '/Data/dec/DOLOS/dolos_timestamps.csv'
out_dir = split

os.makedirs(out_dir, exist_ok=True)

print(f"Reading CSV file: {csv_file}")
with open(csv_file, 'r') as f:
    lines = f.readlines()
    lines = [x.split(',') for x in lines]
    
    # Skip header row
    if len(lines) > 0:
        print(f"Header: {lines[0]}")
        lines = lines[1:]

    print(f"Processing {len(lines)} videos")
    
    # Create VidInfo objects with proper time conversion
    vidinfos = []
    for i, x in enumerate(lines):
        try:
            if len(x) >= 4:  # Make sure we have enough columns
                vidinfo = VidInfo(
                    x[0],  # yt_id
                    x[1],  # file_name
                    time_to_seconds(x[2]),  # start_time
                    time_to_seconds(x[3]),  # end_time
                    out_dir
                )
                vidinfos.append(vidinfo)
            else:
                print(f"Warning: Line {i+1} has fewer than 4 columns: {x}")
        except Exception as e:
            print(f"Error processing line {i+1}: {str(e)}")

print(f"Created {len(vidinfos)} valid video info objects")

# Open a file to track failures
bad_files = open(f'bad_files_{split}.txt', 'w')

# Use fewer threads to avoid rate limiting
results = ThreadPool(2).imap_unordered(download, vidinfos)

# Process results
cnt = 0
successful = 0
failed = 0

for r in results:
    cnt += 1
    print(f"{cnt} / {len(vidinfos)} {r}")
    
    if 'ERROR' in r:
        bad_files.write(r + '\n')
        failed += 1
    else:
        successful += 1

bad_files.close()

print(f"Processing complete. Successful: {successful}, Failed: {failed}")